In [11]:
import pandas as pd
import numpy as np
import pypinyin

In [12]:
data = pd.read_csv(filepath_or_buffer='../test.tsv', sep='	', header=None, names=['sentence1', 'sentence2'])
sentence1 = np.asarray(data.iloc[:, 0:1]).tolist()
sentence2 = np.asarray(data.iloc[:, 1:2]).tolist()
len(sentence1), len(sentence2)

(100000, 100000)

In [13]:
labels = pd.read_csv('practise/paddle_baseline_batch128_ernie_gram.csv', header=None, names=['label'])
labels = np.asarray(labels.iloc[:, 0:1]).transpose().tolist()[0]
labels

[0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,


In [14]:
def check_synonyms(s1, s2):
    if len(s1) != len(s2):
        return False
    for i in range(len(s1)):
        if len(s1[i]) == 1 and len(s2[i]) == 1:
            if s1[i][0] != s2[i][0]:
                return False
        elif len(s1[i]) > 1 and len(s2[i]) == 1:
            if not (s2[i][0] in s1[i]):
                return False
        elif len(s2[i]) > 1 and len(s1[i]) == 1:
            if not (s1[i][0] in s2[i]):
                return False
        else:
            flag = False
            for j in range(len(s1[i])):
                if s1[i][j] in s2[i]:
                    flag = True
                    break
            if not flag:
                return False
    return True

In [15]:
count = 0
output = []

for i in range(100000):
    stc1 = str(sentence1[i][0])
    stc2 = str(sentence2[i][0])

    if str(sentence1[i][0]).__contains__('×') or str(sentence2[i][0]).__contains__('×'):
        labels[i] = 0
        output.append(str(stc1) + '\t' + str(stc2) + '\t' + str(labels[i]))
    else:
        s1 = [pypinyin.pinyin(word, heteronym=True, style=pypinyin.STYLE_NORMAL)[0] for word in sentence1[i][0]]
        # s2 = [pypinyin.pinyin(word, heteronym=True, style=pypinyin.STYLE_NORMAL)[0] for word in sentence2[i][0]]

        if len(stc1) == len(stc2):
            for j in range(len(stc1)):
                temp = stc2[j:] + stc2[:j]
                s2 = [pypinyin.pinyin(word, heteronym=True, style=pypinyin.STYLE_NORMAL)[0] for word in temp]
                if check_synonyms(s1, s2):
                    # print(str(stc1) + '\t' + str(stc2) + '\t' + str(results[i][0]))
                    count += 1
                    output.append(str(stc1) + '\t' + str(stc2) + '\t' + str(labels[i]))
                    labels[i] = 1
                    break

    # if check_synonyms(s1, s2):
    #     # print(str(i + 1) + "\t" + sentence1[i][0] + "\t" + sentence2[i][0])
    #     results[i][0] = 1
    #     count += 1

print('测试集同音率：{}%；同音数：{}'.format(str(count / 100000 * 100), count))

for i in range(100000):
    if str(sentence1[i][0]).__contains__('×') or str(sentence2[i][0]).__contains__('×'):
        labels[i] = 0

with open('result/test_b/synonyms_list.txt', 'w', encoding='utf8', newline='\n') as f:
    for row in output:
        f.write(row + '\n')

with open(file='result/pinyin_misspelling_result_B.csv', mode='w', newline='\n', encoding='utf8') as f:
    for result in labels:
        f.write(str(result) + '\n')

测试集同音率：4.422000000000001%；同音数：4422


In [1]:
import torch

t = torch.randn(2, 2, 768)
index = torch.LongTensor([j for j in range(768 // 8)])
for i in range(len(t.shape) - len(index.shape)):
    index = torch.unsqueeze(index, 0)
index = index.repeat(t.shape[0], t.shape[1], 1)
print(t)
print(torch.gather(t, -1, index))

tensor([[[-0.2612,  0.4183, -0.4772,  ...,  2.1578,  0.7929, -0.6297],
         [ 1.2110,  2.4494, -0.9218,  ..., -0.0633, -0.6502,  0.4937]],

        [[-2.0419, -0.1168,  1.4402,  ..., -0.2512, -0.7106,  0.3480],
         [-1.3805,  0.3210,  0.0187,  ..., -1.2867, -0.8113,  0.2583]]])
tensor([[[-2.6115e-01,  4.1828e-01, -4.7717e-01, -8.9844e-02,  1.5340e-01,
           6.4570e-01,  1.1492e+00, -1.4130e+00,  4.7688e-01, -4.8074e-01,
          -1.2255e+00, -1.2598e-01, -4.3026e-01,  8.8944e-01,  3.9290e-01,
          -2.8022e+00, -8.1574e-01, -4.9936e-01,  9.5791e-01, -1.2399e+00,
          -2.9465e-01, -1.0076e+00,  1.3196e+00,  1.0353e+00,  1.4676e+00,
           1.3698e-01,  1.3393e+00, -1.7505e+00, -7.2276e-01, -9.3909e-02,
          -9.7429e-01, -2.7154e-01,  2.1744e+00, -3.6751e-01, -2.0378e+00,
          -4.3832e-01,  9.0376e-01,  1.2823e-01,  2.7717e+00,  3.9006e-02,
           9.1981e-01,  6.5431e-01, -1.0346e-01,  5.2209e-01,  3.9248e-01,
          -6.7059e-01, -8.6908e-01, -